In [ ]:
# !pip install transformers datasets accelerate torch pandas -q

import pandas as pd
import ast  # For safely evaluating string-lists
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer

: 

In [2]:
# Define the path to your dataset in Kaggle
file_path = '/kaggle/input/3a2mext/3A2M_EXTENDED.csv'

# Load the full dataset
print("Loading full dataset...")
df = pd.read_csv(file_path)
print(f"Loaded {len(df)} total recipes.")

# We only need these three columns
df_clean = df[['title', 'NER', 'directions']].dropna().reset_index(drop=True)
print(f"Found {len(df_clean)} complete (non-null) recipes.")

# --- Create a smaller sample for faster development ---
# Adjust this number based on your Kaggle instance's performance
SAMPLE_SIZE = 100000 
df_sample = df_clean.sample(n=min(SAMPLE_SIZE, len(df_clean)), random_state=42)
print(f"Using a sample of {len(df_sample)} recipes for preprocessing.")

Loading full dataset...
Loaded 2231143 total recipes.
Found 2231142 complete (non-null) recipes.
Using a sample of 100000 recipes for preprocessing.


In [3]:
def format_recipe(row):
    """
    Safely parses and formats a single row into our structured text format.
    """
    try:
        title = row['title']
        
        # Safely parse the 'NER' string-list into a Python list
        ingredients_list = ast.literal_eval(row['NER'])
        # Join the list into a single comma-separated string
        ingredients_str = ", ".join(ingredients_list)
        
        # Safely parse the 'directions' string-list
        directions_list = ast.literal_eval(row['directions'])
        # Join the steps with a newline character
        directions_str = "\n".join(directions_list)
        
        # Return None if any part is empty after parsing
        if not title or not ingredients_str or not directions_str:
            return None
            
        # This is the final format our model will learn
        return f"TITLE: {title}\nINGREDIENTS: {ingredients_str}\nRECIPE:\n{directions_str}"
        
    except (ValueError, SyntaxError):
        # Handle cases where ast.literal_eval fails (e.g., malformed string)
        return None

print("Formatting data into structured text...")
df_sample['text'] = df_sample.apply(format_recipe, axis=1)

# Drop any rows that failed parsing (returned None)
df_processed = df_sample.dropna(subset=['text']).reset_index(drop=True)

print(f"Successfully processed {len(df_processed)} recipes.")
print("\n--- Example of Formatted Text ---")
print(df_processed['text'][0])

Formatting data into structured text...
Successfully processed 88101 recipes.

--- Example of Formatted Text ---
TITLE: Sweet Potato Pecan Salad
INGREDIENTS: stalks celery, sweet potatoes, red onion, olive oil, hanout, fresh rosemary, Kosher salt, green grapes, pecans, pomegranate seeds, scallions, golden raisins, Sherry vinegar
RECIPE:
Preheat the oven to 400 degrees F. Place a baking sheet in the oven to heat.
Toss the celery, sweet potatoes and red onions with the olive oil, ras el hanout and rosemary in a mixing bowl.
Season with salt and pepper.
Transfer the mixture to the heated baking sheet and roast until the potatoes are tender and caramelized, about 30 minutes.
Let the sweet potato mixture cool to room temperature before proceeding.
(Or, if preparing ahead of time, cool the sweet potato mixture in the fridge, removing it 30 minutes before proceeding to allow it to come to room temperature.)
Toss the sweet potato mixture in a large bowl with the grapes, pecans, pomegranate see

In [ ]:
pip install --upgrade transformers huggingface_hub

In [4]:
# Load the pre-trained GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# GPT-2 needs a padding token. We'll use the end-of-sequence (eos) token.
tokenizer.pad_token = tokenizer.eos_token

class RecipeDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.encodings = {
            "input_ids": [],
            "attention_mask": []
        }
        
        print(f"Tokenizing {len(texts)} texts...")
        for text in texts:
            # Add the eos_token to signal the end of a complete recipe
            tokenized = tokenizer(
                text + tokenizer.eos_token, 
                truncation=True, 
                max_length=self.max_length, 
                padding="max_length"
            )
            self.encodings["input_ids"].append(torch.tensor(tokenized['input_ids']))
            self.encodings["attention_mask"].append(torch.tensor(tokenized['attention_mask']))
        print("Tokenization complete.")

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        # The Trainer expects a dictionary
        item = {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
        }
        
        # For Causal LM, the 'labels' are the same as the 'input_ids'
        # The model learns to predict the next token at each step.
        item['labels'] = item['input_ids'].clone()
        
        return item

# --- Create the training and validation datasets ---

# 1. Split our processed DataFrame text column
train_texts, val_texts = train_test_split(
    df_processed['text'], 
    test_size=0.1, 
    random_state=42
)

# 2. Create the Dataset objects
train_dataset = RecipeDataset(train_texts.tolist(), tokenizer)
val_dataset = RecipeDataset(val_texts.tolist(), tokenizer)

print(f"\nCreated {len(train_dataset)} training examples.")
print(f"Created {len(val_dataset)} validation examples.")

# You can inspect a single item to see the output
print("\n--- Example Dataset Item ---")
item = train_dataset[0]
print("Input IDs shape:", item['input_ids'].shape)
print("Labels shape:", item['labels'].shape)
print("Attention Mask shape:", item['attention_mask'].shape)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Tokenizing 79290 texts...
Tokenization complete.
Tokenizing 8811 texts...
Tokenization complete.

Created 79290 training examples.
Created 8811 validation examples.

--- Example Dataset Item ---
Input IDs shape: torch.Size([512])
Labels shape: torch.Size([512])
Attention Mask shape: torch.Size([512])


In [ ]:
!pip uninstall -y transformers
!pip install --no-cache-dir transformers==4.57.1

In [ ]:
import transformers
print(transformers.__version__)
from transformers import TrainingArguments
print(TrainingArguments)


In [5]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"--- Using device: {device.upper()} ---")

if device == 'cpu':
    print("No GPU found")
    use_fp16 = False
else:
    print("GPU is available")
    use_fp16 = True

# 2. Load the Pre-trained Model
print("Loading pre-trained GPT-2 model...")
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Resize token embeddings to match the tokenizer (which has the pad token)
model.resize_token_embeddings(len(tokenizer))

# --- Move model to the correct device ---
model.to(device)

# 3. Define Training Arguments (using older syntax)
print("Setting up training arguments...")

# Calculate steps per epoch for the older argument style
steps_per_epoch = len(train_dataset) // 2  # Using 2 for per_device_train_batch_size
if steps_per_epoch == 0:
    steps_per_epoch = 1 # Avoid division by zero

print(steps_per_epoch)

training_args = TrainingArguments(
    output_dir='./recipe_model',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy="steps",          # <- correct for this build
    eval_steps=steps_per_epoch,
    save_steps=steps_per_epoch,
    logging_steps=100,
    load_best_model_at_end=True,
    fp16=use_fp16,
    report_to="none"
)

# 4. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer  
)

# 5. Start Training
print("Starting the fine-tuning process...")
trainer.train()

# 6. Save the Final Model and Tokenizer
print("Training complete. Saving the final model...")
final_model_path = './final_recipe_model'
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path) 

print(f"Model and tokenizer saved to {final_model_path}")

2025-10-28 15:37:24.527879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761665844.700743      99 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761665844.750599      99 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


--- Using device: CUDA ---
GPU is available
Loading pre-trained GPT-2 model...


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/tmp/ipykernel_99/184480541.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


Setting up training arguments...
39645
Starting the fine-tuning process...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
39645,0.616000,0.622947


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Training complete. Saving the final model...
Model and tokenizer saved to ./final_recipe_model


In [21]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def generate_recipe(model_path, prompt, max_length=300):
    """
    Loads a fine-tuned GPT-2 model and tokenizer from a specified path
    and generates text based on a given prompt.
    """
    
    print(f"Loading model and tokenizer from {model_path}...")
    
    # Check for GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load the tokenizer and model
    try:
        tokenizer = GPT2Tokenizer.from_pretrained(model_path)
        model = GPT2LMHeadModel.from_pretrained(model_path)
    except Exception as e:
        print(f"Error loading model: {e}")
        print("Please ensure the path is correct and all files (config.json, model.safetensors, etc.) are present.")
        return

    # Set the pad token to the eos token if it's not already set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    # Move model to the selected device
    model.to(device)
    model.eval()  # Set model to evaluation mode (important!)

    print("Model loaded. Encoding prompt...")
    
    # Encode the input prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    print("Generating recipe...")

    # Generate text
    with torch.no_grad():  # Disable gradient calculations for inference
        output_sequences = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            temperature=0.7,       # Makes the output less random
            top_k=50,              # Considers the top 50 most likely words
            top_p=0.95,            # Uses nucleus sampling
            do_sample=True,        # Enables sampling
            num_return_sequences=1,# We just want one recipe
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the generated sequence
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    
    return generated_text

# --- How to use the function ---

# 1. Define the path to your saved model
model_directory = "./final_recipe_model"

# 2. Define your starting prompt
#    (Try to match the format of your training data)
input_prompt = "Simple Chicken and Rice" 

# 3. Generate the recipe
recipe_output = generate_recipe(model_directory, input_prompt, max_length=300)

# 4. Print the result
if recipe_output:
    print("\n--- GENERATED RECIPE ---")
    print(recipe_output)

Loading model and tokenizer from ./final_recipe_model...
Using device: cuda
Model loaded. Encoding prompt...
Generating recipe...

--- GENERATED RECIPE ---
Simple Chicken and Rice
INGREDIENTS: chicken breasts, flour, salt, black pepper, onion, vegetable oil, garlic, mushrooms, water, rice
RECIPE:
Heat oil in a deep-fryer or wok.
Add chicken, mushrooms and onions; cook and stir until chicken is cooked through.
Add remaining ingredients; bring to a boil.
Reduce heat; cover and simmer about 20 minutes or until rice is tender.
Serve with rice.


In [22]:
!zip -r my_model.zip ./final_recipe_model

  adding: final_recipe_model/ (stored 0%)
  adding: final_recipe_model/vocab.json (deflated 68%)
  adding: final_recipe_model/merges.txt (deflated 53%)
  adding: final_recipe_model/tokenizer_config.json (deflated 56%)
  adding: final_recipe_model/special_tokens_map.json (deflated 74%)
  adding: final_recipe_model/config.json (deflated 52%)
  adding: final_recipe_model/generation_config.json (deflated 31%)
  adding: final_recipe_model/model.safetensors (deflated 7%)
  adding: final_recipe_model/training_args.bin (deflated 52%)


In [23]:
!pip install evaluate rouge_score nltk -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 35.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
cudf-polars-cu1

In [24]:
import evaluate
import torch
import ast
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm  # For a nice progress bar

# --- 1. Load Model (if not already loaded) ---
# This assumes 'model' and 'tokenizer' are not in memory
# If they are, you can comment out this block.
print("Loading model and tokenizer...")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_path = './final_recipe_model'

model = GPT2LMHeadModel.from_pretrained(model_path)
model.to(device)
model.eval()
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
print(f"Model loaded and on device: {device}")

# --- 2. Get our Validation Set ---
# We re-create the validation split from our processed dataframe
# to get the original, separated columns.
try:
    _ , val_df = train_test_split(df_processed, test_size=0.1, random_state=42)
except NameError:
    print("Error: 'df_processed' not found. Please re-run the data preparation cells.")

# We'll test on a smaller sample to save time
# 200 is a good number for a solid score.
EVAL_SAMPLES = 200
eval_subset = val_df.sample(n=min(EVAL_SAMPLES, len(val_df)), random_state=42)

predictions = []
references = []

print(f"Generating {len(eval_subset)} predictions for evaluation...")

# --- 3. Generation Loop ---
for _, row in tqdm(eval_subset.iterrows(), total=len(eval_subset)):
    
    # A. Format the Prompt
    title = row['title']
    ingredients_list = ast.literal_eval(row['NER'])
    ingredients_str = ", ".join(ingredients_list)
    prompt_text = f"TITLE: {title}\nINGREDIENTS: {ingredients_str}\nRECIPE:\n"
    
    # B. Generate the Prediction
    input_ids = tokenizer.encode(prompt_text, return_tensors='pt').to(device)
    prompt_length = input_ids.shape[1]
    
    with torch.no_grad():
        # We use beam search for evaluation as it's deterministic and high-quality
        output_sequences = model.generate(
            input_ids=input_ids,
            max_length=prompt_length + 250, # Give it 250 tokens for the recipe
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Decode only the newly generated text
    pred_text = tokenizer.decode(output_sequences[0][prompt_length:], skip_special_tokens=True)
    predictions.append(pred_text)
    
    # C. Format the Reference (Ground Truth)
    ref_list = ast.literal_eval(row['directions'])
    ref_text = "\n".join(ref_list)
    references.append(ref_text)

print("Generation complete. Calculating scores...")

# --- 4. Compute Metrics ---
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

rouge_results = rouge.compute(predictions=predictions, references=references)
bleu_results = bleu.compute(predictions=predictions, references=references)

print("\n--- 📈 ROUGE Results ---")
print(rouge_results)

print("\n--- 📈 BLEU Results ---")
print(bleu_results)

Loading model and tokenizer...
Model loaded and on device: cuda
Generating 200 predictions for evaluation...


  0%|          | 0/200 [00:00<?, ?it/s]

Generation complete. Calculating scores...



--- 📈 ROUGE Results ---
{'rouge1': 0.29556364855290834, 'rouge2': 0.08060470793226145, 'rougeL': 0.19320904225952973, 'rougeLsum': 0.2576366630349671}

--- 📈 BLEU Results ---
{'bleu': 0.048683749850367025, 'precisions': [0.40626372936672317, 0.11148541282654294, 0.03875619648490311, 0.015913389421509164], 'brevity_penalty': 0.6696556509349939, 'length_ratio': 0.713780127228743, 'translation_length': 15933, 'reference_length': 22322}
